In [ ]:
import torch
import json

pth_file_path = "model.pth"

# Load the checkpoint data
checkpoint = torch.load(pth_file_path, map_location=torch.device('cpu'))

# checkpoint is typically a dictionary. Let's see its keys:
print("Keys in the checkpoint:", checkpoint.keys())

# If this .pth file has a model_state_dict (common practice in PyTorch)
if "model_state_dict" in checkpoint:
    model_state = checkpoint["model_state_dict"]
    print("Number of keys in model_state_dict:", len(model_state.keys()))


Create JSON Function

In [ ]:

def load_model_info(pth_file_path):
    """
    Loads a PyTorch .pth checkpoint file and restructures its data into a
    hierarchical dictionary ready for JSON serialization.
    """
    checkpoint = torch.load(pth_file_path, map_location=torch.device('cpu'))
    model_info = {}

    # Access model_state_dict directly, or use the checkpoint if model_state_dict isn't a key
    model_state = checkpoint.get("model_state_dict", checkpoint)

    names_list = ['quant', 'conv1', 'conv2', 'fc1', 'fc2']

    for layer in range(len(names_list)):
        if names_list[layer] == 'quant':
            # Handling the 'quant' layer separately as it has different attributes
            model_info[names_list[layer]] = {
                "scale": float(model_state[f'{names_list[layer]}.scale'].item()),
                "zero_point": int(model_state[f'{names_list[layer]}.zero_point'].item())
            }
        elif (names_list[layer] == 'conv1' or names_list[layer] == 'conv2'):
            # Handling 'conv1' and 'conv2' assuming they are quantized layers
            if f'{names_list[layer]}.weight' in model_state:
                weights = model_state[f'{names_list[layer]}.weight']
                effective_scale = float(model_state[f'{names_list[layer-1]}.scale'])*float(weights.q_scale())
                biases_list = model_state[f'{names_list[layer]}.bias'].tolist()
                # biases = []

                for i in range(len(biases_list)):
                    biases_list[i] = round(float(biases_list[i])/effective_scale)

                model_info[names_list[layer]] = {
                    "weights": weights.int_repr().tolist(),
                    "biases": biases_list,
                    "weights_scale": float(weights.q_scale()),
                    "weights_zero_point": int(weights.q_zero_point()),
                    "layer_scale": float(model_state[f'{names_list[layer]}.scale'].item()),
                    "layer_zero_point": int(model_state[f'{names_list[layer]}.zero_point'].item())
                }

        else:
            if f'{names_list[layer]}._packed_params._packed_params' in model_state:

                weights = model_state[f'{names_list[layer]}._packed_params._packed_params'][0]
                # print(weights)
                effective_scale = float(model_state[f'{names_list[layer-1]}.scale'])*float(weights.q_scale())
                biases_list = model_state[f'{names_list[layer]}._packed_params._packed_params'][1].tolist()
                # print(biases_list)
                for i in range(len(biases_list)):
                    biases_list[i] = round(float(biases_list[i])/effective_scale)


                model_info[names_list[layer]] = {
                    "weights": weights.int_repr().tolist(),
                    "biases": biases_list,
                    "weights_scale": float(weights.q_scale()),
                    "weights_zero_point": int(weights.q_zero_point()),
                    "layer_scale": float(model_state[f'{names_list[layer]}.scale'].item()),
                    "layer_zero_point": int(model_state[f'{names_list[layer]}.zero_point'].item())
                }

    Final_Scales = {}
    for layer in range(len(names_list)):
        if names_list[layer] == 'quant':
            continue
        elif names_list[layer] == 'conv1':
            effective_scale = model_info[names_list[layer]]['weights_scale'] * model_info[names_list[layer-1]]['scale']
            final_scale = round((effective_scale*2**16)/model_info[names_list[layer]]['layer_scale'])
            Final_Scales[names_list[layer]] =  final_scale
            print(Final_Scales[names_list[layer]])
        else:
            effective_scale = model_info[names_list[layer]]['weights_scale'] * model_info[names_list[layer-1]]['layer_scale']
            final_scale =round((effective_scale*2**16)/model_info[names_list[layer]]['layer_scale'])
            Final_Scales[names_list[layer]] = final_scale

    model_info['Final_Scales'] = Final_Scales
    model_info['label_mapping'] = checkpoint.get("labels_mapping", checkpoint)


    return model_info


Create JSON

In [ ]:
if __name__ == "__main__":
    # Replace this with the path to your .pth file
    pth_file_path = "model.pth"

    # Build the model info dictionary
    model_info_dict = load_model_info(pth_file_path)
    print(model_info_dict.keys())

    # Convert to JSON string
    model_info_json = json.dumps(model_info_dict, indent=4)

    # Save to a file
    with open("model_info.json", "w") as f:
        f.write(model_info_json)

    # Print to console (for demonstration)
    print("JSON data has been saved to 'model_info.json'.")